# Ampliando
En esta lección generaremos correos electrónicos de atención al cliente adaptados a la revisión de cada cliente.

In [1]:
import openai
import os
from IPython.display import Markdown, HTML
import requests
import json

from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
auth_token = os.getenv("AUTH_TOKEN")


headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {auth_token}"
}

url = 'https://api.awanllm.com/v1/chat/completions' # URL for the API
# Función para llamar a la API de LLMCloud para preguntarle a LLAMA
def preguntar_llama(sys_prompt, prompt):
    payload = json.dumps({
        "model": "Meta-Llama-3-8B-Instruct",
        "messages": [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": prompt}
        ],
      })
    return requests.request("POST", url, headers=headers, data=payload).json()['choices'][0]['message']['content']

client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0.1):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

## Personalizar la respuesta automática al correo electrónico de un cliente

In [2]:
# Teniendo en cuenta lo que aprendimos en la lección sobre "inferir" el sentimiento del cliente original, personaliza el correo electrónico

sentiment = "negative"

# reseña de una licuadora
review = """
Buenos días,

Me gustaría compartir mi experiencia con la licuadora que compré en su tienda. La licuadora es muy bonita y elegante, pero no es tan potente como esperaba. La primera vez que la usé, se calentó mucho y no pude hacer un batido. Además, la tapa no encaja bien y se sale cuando la licuadora está en funcionamiento.

Entonces, en noviembre, todavía tenían el sistema de 17 piezas en oferta por alrededor de $49, aproximadamente la mitad de precio. Sin embargo, por alguna razón, alrededor de la segunda semana de diciembre, los precios subieron a entre $70-$89 por el mismo sistema. El sistema de 11 piezas también aumentó en alrededor de $10 o más desde el precio de venta anterior de $29. La base, donde se encaja la cuchilla, no se ve tan bien como en ediciones anteriores de hace unos años, pero planeo ser muy cuidadoso con ella.

Un consejo especial al hacer batidos es cortar finamente y congelar las frutas y verduras que planeas usar, de esa manera puedes evitar agregar tanto hielo al hacer tu batido. Después de aproximadamente un año, el motor comenzó a hacer un ruido extraño. Llamé al servicio al cliente, pero la garantía ya había expirado, así que tuve que comprar otro.

PD: La calidad general ha disminuido en este tipo de productos, por lo que están contando en gran medida con el reconocimiento de marca y la lealtad del consumidor para mantener las ventas. El producto llegó en aproximadamente dos días.

Atentamente,
Alfonso
"""

In [3]:
prompt = f"""
Eres un asistente de IA de servicio al cliente.
Tu tarea es enviar una respuesta por correo electrónico a un cliente valioso.
Dado el correo electrónico del cliente delimitado por ```, genera una respuesta para agradecer al cliente por su reseña.
Si el sentimiento es positivo o neutral, agradéceles por su reseña.
Si el sentimiento es negativo, discúlpate y sugiere que pueden comunicarse con el servicio al cliente.
Asegúrate de usar detalles específicos de la reseña.
Escribe en un tono conciso y profesional.
Firma el correo electrónico como `Agente de servicio al cliente de IA`.
Reseña del cliente: ```{review}```
Sentimiento de la reseña: {sentiment}
"""
response = get_completion(prompt)
display(Markdown(f"### Respuesta de GPT\n{response}"))
print("\n---------------------------------------------\n")
display(Markdown(f"### Respuesta de LLAMA\n{preguntar_llama('Eres un asistente de IA cuyo trabajo es responder a lo que te está pidiendo el usuario de la forma más fiel al prompt sin preámbulos explicativos ni resúmenes finales.', prompt)}"))

### Respuesta de GPT
Estimado Alfonso,

Gracias por tomarte el tiempo de compartir tu experiencia con la licuadora que adquiriste en nuestra tienda. Lamentamos mucho escuchar que no cumplió con tus expectativas y que experimentaste problemas con su funcionamiento y durabilidad.

Nos disculpamos por los inconvenientes que esto te haya causado. Valoramos tus comentarios y te aseguramos que tomaremos en consideración tus observaciones para mejorar nuestros productos y servicios.

Si necesitas alguna asistencia adicional o deseas discutir más a fondo tus inquietudes, no dudes en comunicarte con nuestro equipo de servicio al cliente. Estamos aquí para ayudarte en lo que necesites.

Agradecemos tu retroalimentación y la oportunidad de mejorar. Esperamos poder brindarte una mejor experiencia en el futuro.

Atentamente,
Agente de servicio al cliente de IA


---------------------------------------------



### Respuesta de LLAMA
Asunto: Agradezco su reseña y le ofrezco ayuda adicional

Querido Alfonso,

Gracias por tomarse el tiempo de compartir su experiencia con nuestra licuadora en nuestra tienda. Me duele ver que no haya cumplido con sus expectativas en cuanto a la potencia y la calidad. Lo siento especialmente por el problema con la tapa y el ruido extraño después de un año de uso.

En particular, me gustó saber que valoró nuestro sistema de 17 piezas y nos hizo notar el aumento en precios en cierto momento. Estoy de acuerdo con usted en que la base de la cuchilla no parece tan buena como en versiones anteriores.

Por supuesto, lamentamos que la garantía hubiera expirado antes de que pudieran ayudarnos con el problema con el motor. Si necesita ayuda adicional o tiene alguna pregunta, no dude en ponerse en contacto con nosotros. Nuestro equipo de servicio al cliente estará encantado de asistirle.

Agradezco su honesta retroalimentación y esperamos mejorar nuestros productos y servicios para ofrecer una mejor experiencia al futuro.

Atentamente,
Agente de servicio al cliente de IA

## Recordándole al modelo que utilice detalles del email del cliente

In [4]:
prompt = f"""
Eres un asistente de IA de servicio al cliente.
Tu tarea es enviar una respuesta por correo electrónico a un cliente valioso.
Dado el correo electrónico del cliente delimitado por ```, genera una respuesta para agradecer al cliente por su reseña.
Si el sentimiento es positivo o neutral, agradéceles por su reseña.
Si el sentimiento es negativo, discúlpate y sugiere que pueden comunicarse con el servicio al cliente.
Asegúrate de usar detalles específicos de la reseña para que la respuesta no sea tan robótica.
Escribe en un tono conciso y profesional.
Firma el correo electrónico como `Agente de servicio al cliente de IA`.
Reseña del cliente: ```{review}```
Sentimiento de la reseña: {sentiment}
"""
response = get_completion(prompt, temperature=1)
display(Markdown(f"### Respuesta de GPT\n{response}"))
print("\n---------------------------------------------\n")
display(Markdown(f"### Respuesta de LLAMA\n{preguntar_llama('Eres un asistente de IA cuyo trabajo es responder a lo que te está pidiendo el usuario de la forma más fiel al prompt sin preámbulos explicativos ni resúmenes finales.', prompt)}"))

### Respuesta de GPT
Estimado Alfonso,

Gracias por tomarte el tiempo de compartir tu experiencia con la licuadora que adquiriste en nuestra tienda. Lamentamos escuchar que no cumplió completamente tus expectativas y que experimentaste problemas con su funcionamiento.

Nos disculpamos por las molestias ocasionadas por la falta de potencia de la licuadora, así como por los problemas con la tapa y el sobrecalentamiento que mencionaste. También lamentamos la situación con el cambio de precios en nuestros productos y la disminución en la calidad percibida.

Apreciamos tus comentarios detallados y tu consejo sobre el uso de frutas y verduras congeladas para los batidos. Tus observaciones son muy valiosas para nosotros y las tendremos en cuenta mientras trabajamos en mejorar nuestros productos y servicios.

Si necesitas alguna asistencia adicional o deseas compartir más detalles sobre tu experiencia, por favor no dudes en contactarnos a través de nuestro servicio al cliente. Estamos aquí para ayudarte en lo que necesites.

Agradecemos tu sinceridad y la oportunidad de mejorar nuestros productos. Valoramos tu lealtad como cliente y esperamos poder brindarte una mejor experiencia en el futuro.

Atentamente,
Agente de servicio al cliente de IA


---------------------------------------------



### Respuesta de LLAMA
Asunto: Agradezco su retroalimentación sobre nuestra licuadora

Estimado Sr. Alfonso,

Queremos expresar nuestro agradecimiento por haber tomado el tiempo de compartir su experiencia con nosotros sobre la licuadora que compró en nuestra tienda. Me entristece saber que no estuvo satisfecho con el rendimiento de la licuadora, especialmente con la potencia y la calidad de la tapa.

Disculpamos cualquier incomodidad causada por el aumento de precio del sistema de piezas y lamentamos que no haya podido aprovechar plenamente su compra debido a la caducidad de la garantía. Estamos trabajando para mejorar constantemente nuestros productos y servicios.

Agradezco también sus sugerencias prácticas para mejorar el uso de la licuadora, como cortar finamente y congelar las frutas y verduras antes de hacer batidos. Estas recomendaciones serán útiles para otros clientes.

Gracias nuevamente por su retroalimentación. Si necesita ayuda adicional o tiene alguna pregunta, por favor no dude en ponerse en contacto con nuestro servicio al cliente.

Atentamente,
Agente de servicio al cliente de IA